**Turning OFF AQE and DPP and AutoBroadcast**

In [0]:
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")


In [0]:
df = spark.read.format("csv")\
        .option("header",True)\
        .option("inferSchema",True)\
        .load("/FileStore/rawdata/BigMart_Sales.csv")

df = df.limit(8)

In [0]:
df.display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636


**Preparing the Partitioned Data**

In [0]:
df.write.format("parquet")\
        .mode("append")\
        .partitionBy("Item_identifier")\
        .option("path","/FileStore/rawdata/dpp_partionednew")\
        .save()

**Non Partitioned Data**

In [0]:
df.write.format("parquet")\
        .mode("append")\
        .option("path","/FileStore/rawdata/dpp_nonpartioned")\
        .save()

**Dataframes**

In [0]:
df1 = spark.read.format("parquet")\
          .load("/FileStore/rawdata/dpp_partionednew")

In [0]:
df2 = spark.read.format("parquet")\
          .load("/FileStore/rawdata/dpp_nonpartioned")

**JOINS**

In [0]:
from pyspark.sql.functions import *

In [0]:
df_join = df1.join(df2.filter(col("Item_Identifier")=="FDA15"),df1['Item_Identifier']==df2['Item_Identifier'],"inner")

In [0]:
df_join.display()

Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_identifier,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,FDA15,FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138


In [0]:
df_join.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) BroadcastHashJoin [Item_Identifier#118], [Item_Identifier#131], Inner, BuildRight, false, true
   :- *(2) ColumnarToRow
   :  +- FileScan parquet [Item_Weight#107,Item_Fat_Content#108,Item_Visibility#109,Item_Type#110,Item_MRP#111,Outlet_Identifier#112,Outlet_Establishment_Year#113,Outlet_Size#114,Outlet_Location_Type#115,Outlet_Type#116,Item_Outlet_Sales#117,Item_identifier#118] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/rawdata/dpp_partionednew], PartitionFilters: [isnotnull(Item_identifier#118), (Item_identifier#118 = FDA15)], PushedFilters: [], ReadSchema: struct<Item_Weight:double,Item_Fat_Content:string,Item_Visibility:double,Item_Type:string,Item_MR...
   +- ShuffleQueryStage 0, Statistics(sizeInBytes=176.0 B, rowCount=1, isRuntime=true)
      +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=187]
         +- *(1) Filter (isnotnull